# Introduction

Starbucks are looking to find the next store to turn into a [Starbucks Reserve Roastery](https://www.businessinsider.com/starbucks-reserve-roastery-compared-regular-starbucks-2018-12#also-on-the-first-floor-was-the-main-coffee-bar-five-hourglass-like-units-hold-the-freshly-roasted-coffee-beans-that-are-used-in-each-order-the-selection-rotates-seasonally-5). These roasteries are much larger than a typical Starbucks store and have several additional features, including various food and wine options, along with upscale lounge areas.  You'll investigate the demographics of various counties in the state of California, to determine potentially suitable locations.

In [107]:
import math
import pandas as pd
import geopandas as gpd
from geopandas.tools import geocode

import folium 
from folium import Marker
from folium.plugins import MarkerCluster

In [108]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

## Import the store locations dataset

The dataset contains locations of Starbucks in the state of California

In [109]:
starbucks = pd.read_csv('data/starbucks_locations.csv')

In [110]:
starbucks.head()

,Store Number,Store Name,Address,City,Longitude,Latitude
0,10429-100710,Palmdale & Hwy 395,14136 US Hwy 395 Adelanto CA,Adelanto,-117.40,34.51
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15
3,29839-255026,Target Anaheim T-0677,8148 E SANTA ANA CANYON ROAD AHAHEIM CA,AHAHEIM,-117.75,33.87
4,23463-230284,Safeway - Alameda 3281,2600 5th Street Alameda CA,Alameda,-122.28,37.79


In [111]:
starbucks.shape

(2821, 6)

There are total number of 2821 store locations

## Check for missing Values

In [112]:
starbucks.isna().sum()

Store Number    0
Store Name      0
Address         0
City            0
Longitude       5
Latitude        5
dtype: int64

Identify which cities have missing Longitude and Latitude

In [113]:
rows_with_missing = starbucks[starbucks['Longitude'].isna()]
rows_with_missing

,Store Number,Store Name,Address,City,Longitude,Latitude
153,5406-945,2224 Shattuck - Berkeley,2224 Shattuck Avenue Berkeley CA,Berkeley,NaN,NaN
154,570-512,Solano Ave,1799 Solano Avenue Berkeley CA,Berkeley,NaN,NaN
155,17877-164526,Safeway - Berkeley #691,1444 Shattuck Place Berkeley CA,Berkeley,NaN,NaN
156,19864-202264,Telegraph & Ashby,3001 Telegraph Avenue Berkeley CA,Berkeley,NaN,NaN
157,9217-9253,2128 Oxford St.,2128 Oxford Street Berkeley CA,Berkeley,NaN,NaN


Seems that all of the records with missing Longitude and Latitude are in the city of Berkeley.

## Use OpenStreetMap Nominatim geocoder

Using nominatim and the address and longitude and latitude for each of these locations

In [114]:
def get_geocode(address):
    point = geocode(address, provider='nominatim').geometry[0]
    return pd.Series({'Longitude': point.x, 'Latitude': point.y})

In [115]:
updated_locations = rows_with_missing.apply(lambda x: get_geocode(x['Address']), axis=1)
updated_locations

,Longitude,Latitude
153,-122.268230,37.868839
154,-122.280009,37.891471
155,-122.269869,37.881177
156,-122.259370,37.855947
157,-122.266095,37.870253


In [116]:
starbucks.update(updated_locations)

In [117]:
type(result.iloc[0].geometry)

shapely.geometry.point.Point

In [118]:
# Create a base map
m = folium.Map(location=[37.88,-122.26], zoom_start=13)

#Add a marker for each Berkeley location
for idx, row in starbucks[starbucks['City']=="Berkeley"].iterrows():
    Marker([row['Latitude'], row['Longitude']]).add_to(m)

# Uncomment to see a hint
#q_2.a.hint()

# Show the map
embed_map(m, 'q_2.html')

All 5 locations seem to be around the Berkeley area. So we can conclude that the geocoding worked and got the correct coordinates

## Map All locations

In [119]:
# Create a base map
m2 = folium.Map(location=[36.77,-119.4], zoom_start=6)


#Add a marker for each Berkeley location
for i in range(0,len(starbucks)):
    folium.CircleMarker([starbucks.iloc[i]['Latitude'],starbucks.iloc[i]['Longitude']], radius=1, fill=True).add_to(m2)

# Uncomment to see a hint
#q_2.a.hint()

# Show the map
embed_map(m2, 'q_2.html')

## Examine Demographic Data

### Import County Boundries

In [120]:
CA_counties = gpd.read_file("data/CA_county_boundaries/CA_county_boundaries.shp")
CA_counties.head()

,GEOID,name,area_sqkm,geometry
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6..."
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7..."
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822..."
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3..."
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360..."


### Import Demographic Data

In [121]:
CA_pop = pd.read_csv("data/CA_county_population.csv", index_col="GEOID")
CA_high_earners = pd.read_csv("data/CA_county_high_earners.csv", index_col="GEOID")
CA_median_age = pd.read_csv("data/CA_county_median_age.csv", index_col="GEOID")

In [122]:
CA_pop.head()

,population
GEOID,
6001,1666753
6003,1101
6005,39383
6007,231256
6009,45602


Each table has the relevant attribute for the associated GEOID

## Join the Tables

In [123]:
merged_demographic = CA_pop.join([CA_high_earners, CA_median_age]).reset_index()
merged_demographic.head()

,GEOID,population,high_earners,median_age
0,6001,1666753,145696,37.3
1,6003,1101,30,44.9
2,6005,39383,1220,50.6
3,6007,231256,6860,36.9
4,6009,45602,2046,51.6


In [124]:
CA_merged = pd.merge(CA_counties, merged_demographic, on='GEOID', how='left')

In [125]:
CA_merged.isna().sum()

GEOID           0
name            0
area_sqkm       0
geometry        0
population      0
high_earners    0
median_age      0
dtype: int64

In [126]:
CA_merged.shape

(58, 7)

### Get population Density

In [127]:
CA_merged['density'] = CA_merged['population']/CA_merged['area_sqkm']

## Identify promising Counties

The criteria for a successful county has been decided as follows:


In [128]:
sel_counties = CA_merged[((CA_merged.high_earners > 100000) &
                         (CA_merged.median_age < 38.5) &
                         (CA_merged.density > 285) &
                         ((CA_merged.median_age < 35.5) |
                         (CA_merged.density > 1400) |
                         (CA_merged.high_earners > 500000)))]

In [129]:
sel_counties.head()

,GEOID,name,area_sqkm,geometry,population,high_earners,median_age,density
5,6037,Los Angeles County,12305.376879,"MULTIPOLYGON (((-118.66761 33.47749, -118.6682...",10105518,501413,36.0,821.227834
8,6073,San Diego County,11721.342229,"POLYGON ((-117.43744 33.17953, -117.44955 33.1...",3343364,194676,35.4,285.237299
10,6075,San Francisco County,600.588247,"MULTIPOLYGON (((-122.60025 37.80249, -122.6123...",883305,114989,38.3,1470.733077


## Identify Promising Stores

Now that the counties to focus on have been identified, we need to identify which stores within the counties are suitable candidates. 

Let's create a GeoDataFrame with all the starbucks locations.

In [130]:
starbucks_gdf = gpd.GeoDataFrame(starbucks, geometry=gpd.points_from_xy(starbucks.Longitude, starbucks.Latitude))
starbucks_gdf.crs = {'init': 'epsg:4326'}

/Users/numalj/opt/miniconda3/envs/minimal_ds/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [131]:
starbucks_gdf.head()

,Store Number,Store Name,Address,City,Longitude,Latitude,geometry
0,10429-100710,Palmdale & Hwy 395,14136 US Hwy 395 Adelanto CA,Adelanto,-117.40,34.51,POINT (-117.40000 34.51000)
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16,POINT (-118.76000 34.16000)
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15,POINT (-118.76000 34.15000)
3,29839-255026,Target Anaheim T-0677,8148 E SANTA ANA CANYON ROAD AHAHEIM CA,AHAHEIM,-117.75,33.87,POINT (-117.75000 33.87000)
4,23463-230284,Safeway - Alameda 3281,2600 5th Street Alameda CA,Alameda,-122.28,37.79,POINT (-122.28000 37.79000)


In [132]:
# Verify that it matches the crs of the demographics table

In [133]:
CA_merged.geometry.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [134]:
# Number of stores in the selected counties
locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties, op='within')
num_stores = len(locations_of_interest)
num_stores

/Users/numalj/opt/miniconda3/envs/minimal_ds/lib/python3.8/site-packages/geopandas/tools/sjoin.py:58: UserWarning: CRS of frames being joined does not match!(+init=epsg:4326 +type=crs != epsg:4326)
  warn(


1043

### Visualize the results

In [139]:
# Create a base map
m3 = folium.Map(location=[35,-120], zoom_start=6)

#Add a marker for each selected location
for i in range(0,len(locations_of_interest)):
    folium.CircleMarker([locations_of_interest.iloc[i]['Latitude'],locations_of_interest.iloc[i]['Longitude']], radius=1, fill=True).add_to(m3)

embed_map(m3, 'q_3.html')